<a href="https://colab.research.google.com/github/saipklvs/SCL/blob/master/TextGenerationRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf 
import numpy as np 
import os 
import time 


In [0]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')


In [16]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('length of the text: {} characters'.format(len(text)))

length of the text: 1115394 characters


# RNN architectire 



The below we will be seeing the text and implementing the RNN architecture 

Step 1:
  Pre Procee of the text:
    Map the text to the numbers and numbers to the text,
    so that we will have the integer representation for each character 

---
Step 2:
  Prediction Task:
    Give a character, or a sequence of characters what will be the next 
    task. the input will be a sequence of characters and the output will be 
    an character which we predict at each time step 

---
Step 3:
  Creating the training examples and targets:
    Divide the text in to exampl sequencesm each input contains the 
    seq length from the text. For each input sequence, the corresponding 
    targets contain the same length of the text, except shifted by one 
    character to the right 
  





In [17]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [0]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [19]:
seq_length = 100
examples_per_epoch = len(text) ## sequence length + 1

## Create the training examples and targets 
char_dataset  = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

F
i
r
s
t


In [20]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [0]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


# Model Building
## Main layers in the building the model 

Step 1:
  tf.keras.layers.Embedding:
    a. the input layer; A trainable look up table that will map the numbers 
    of each charatcer to a vector with embedding_dim dimensions;


---


  tf.keras.layers.GRU:
    a. A type of RNN with size units=rnn_units


---


  tf.keras.layers.Dense:
    a. The output layer with vocab_size outputs

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024